In [39]:
# https://zhuanlan.zhihu.com/p/90487361
# https://www.quantconnect.com/research/15258/dual-thrust-trading-algorithm/p1

import duckdb
con = duckdb.connect(':memory:')

In [41]:
sql = """
CREATE TABLE market_data AS
FROM 'bitcoin_2024-01-01_2024-09-27.csv' t
"""
con.execute(sql)

In [42]:
sql = """
select t.* 
from market_data t order by Start limit 10
"""
con.sql(sql)

┌────────────┬────────────┬──────────┬──────────┬──────────┬──────────┬────────────────────┬───────────────────┐
│   Start    │    End     │   Open   │   High   │   Low    │  Close   │       Volume       │    Market Cap     │
│    date    │    date    │  double  │  double  │  double  │  double  │       double       │      double       │
├────────────┼────────────┼──────────┼──────────┼──────────┼──────────┼────────────────────┼───────────────────┤
│ 2024-01-01 │ 2024-01-02 │ 42276.34 │ 44122.03 │ 42227.99 │ 44122.03 │  25446134905.26736 │ 838250699610.9688 │
│ 2024-01-02 │ 2024-01-03 │  44179.5 │ 45846.21 │  44179.5 │ 44957.63 │  46593616092.15278 │ 885906624909.5868 │
│ 2024-01-03 │ 2024-01-04 │ 44944.83 │ 45474.95 │ 40931.46 │ 42818.03 │  99124375067.69098 │ 860210389676.8438 │
│ 2024-01-04 │ 2024-01-05 │ 42845.26 │ 44664.55 │ 42635.44 │ 44188.15 │ 163905349005.13196 │ 851366855527.3577 │
│ 2024-01-05 │ 2024-01-06 │ 44168.03 │ 44229.51 │ 43142.74 │ 44112.11 │  142446432011.0521 │  85

In [9]:
N = 5
K1 = 0.2
K2 = 0.3

In [45]:
sql = """
WITH dual_thrust AS (
    SELECT *, IF(HH - LC > HC - LL, HH - LC, HC - LL) as range
    FROM(
        SELECT
            *,
            max(High) OVER (ORDER BY Start ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS HH,
            max(Close) OVER (ORDER BY Start ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS HC,
            min(Close) OVER (ORDER BY Start ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS LC,
            max(Low) OVER (ORDER BY Start ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS LL
        FROM
            market_data
    )
)
SELECT *, Open, Open + lead(0.2 * range) OVER validity as buy_trigger, Open - lead(0.3 * range) OVER validity as sell_trigger
FROM dual_thrust
WINDOW validity AS (ORDER BY Start desc)
ORDER BY Start desc
"""
con.sql(sql)

┌────────────┬────────────┬──────────┬──────────┬──────────┬──────────┬────────────────────┬────────────────────┬──────────┬──────────┬──────────┬──────────┬────────────────────┬──────────┬────────────────────┬────────────────────┐
│   Start    │    End     │   Open   │   High   │   Low    │  Close   │       Volume       │     Market Cap     │    HH    │    HC    │    LC    │    LL    │       range        │   Open   │    buy_trigger     │    sell_trigger    │
│    date    │    date    │  double  │  double  │  double  │  double  │       double       │       double       │  double  │  double  │  double  │  double  │       double       │  double  │       double       │       double       │
├────────────┼────────────┼──────────┼──────────┼──────────┼──────────┼────────────────────┼────────────────────┼──────────┼──────────┼──────────┼──────────┼────────────────────┼──────────┼────────────────────┼────────────────────┤
│ 2024-09-26 │ 2024-09-27 │ 63140.63 │ 65616.04 │ 62792.05 │ 65142.08 │ 

In [38]:
con.close()